In [210]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [211]:
df = pd.read_csv('taxi_dataset_with_target.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925


In [212]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1458644 entries, id2875421 to id1209952
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   vendor_id           1458644 non-null  int64  
 1   pickup_datetime     1458644 non-null  object 
 2   passenger_count     1458644 non-null  float64
 3   store_and_fwd_flag  1458644 non-null  int64  
 4   trip_duration       1458644 non-null  float64
 5   distance_km         1458644 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 77.9+ MB


In [213]:
# df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [214]:
# df['year'] = df['pickup_datetime'].dt.year

In [215]:
# df['month'] = df['pickup_datetime'].dt.month

Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [216]:
from sklearn.linear_model import LinearRegression

### Your code is here

In [217]:
X = df.drop(['trip_duration', 'pickup_datetime'], axis=1)
Y = df['trip_duration']

In [218]:
X.shape

(1458644, 4)

In [219]:
model=LinearRegression()
model.fit(X, Y)

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [220]:
### Your code is here
for column, coef in zip(X.columns, model.coef_):
    print(column, coef)
print()
print(*[round(c, 3) for c in model.coef_], round(model.intercept_, 3), sep =',')

vendor_id 198.46313674390353
passenger_count 0.29631294795221347
store_and_fwd_flag 56.46912165440558
distance_km 115.27353763047665

198.463,0.296,56.469,115.274,171.657


In [ ]:
model.coef_.tolist(), model.intercept_

In [221]:
### Полученные коэффициенты будут являться ответом на задание 5 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив в большую сторону до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [222]:
import numpy as np

### Your code is here

def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
        """
        :param X: матрица объектов
        :param Y: вектор (матрица с 1 столбцом) ответов
        :param fit_intercept: добавить ли константную переменную в данные?
        
        :return: одномерный numpy-array с полученными коэффициентами beta
        """
        if fit_intercept:
            b = np.ones((X.shape[0],1), dtype=np.int64)   
            X = np.column_stack((X, b))
            #print(X)
        d = np.dot(X.T, X)
        o = np.linalg.inv(d)
        f = np.dot(o, X.T)
        betas = np.dot(f, Y)
        
        return betas

In [ ]:
def LinearRegressionByMatrix(X, Y, fit_intercept=True):
    
    X = np.c_[ X, np.ones(X.shape[0]) ]
    
    return np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)

In [223]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

betas = LinearRegressionByMatrix(X_train, Y_train)
betas

array([198.46313674,   0.29631295,  56.46912165, 115.27353763,
       171.65658146])

In [224]:
[round(c, 3) for c in betas]

[198.463, 0.296, 56.469, 115.274, 171.657]

In [225]:
### Полученные коэффициенты будут являться ответом на задание 6 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив в большую сторону до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [ ]:
np.dot(np.c_[ X_train, np.ones(X_train.shape[0]) ], betas)

In [226]:
### Your code is here
b = np.ones((X.shape[0],1), dtype=np.int64)   
X_test = np.column_stack((X_train, b))
y_test = np.dot(X_test, betas)
y_test

array([ 818.7747282 ,  655.65912268, 1382.6469154 , ..., 1548.74134353,
        573.4306718 ,  578.2338068 ])

In [227]:
model.predict(X)

array([ 818.7747282 ,  655.65912268, 1382.6469154 , ..., 1548.74134353,
        573.4306718 ,  578.2338068 ])

In [228]:
#MSE
(((y_test - Y)**2).mean())

27173382.5775981

In [229]:
#MSE
from sklearn.metrics import mean_squared_error
mean_squared_error(Y, y_test)

27173382.577599242